In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# Any results you write to the current directory are saved as output.
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
from tqdm import tqdm_notebook as tqdm
import math
import torch.nn.functional as F
from torch.nn import init

C:\Users\User\Anaconda3\envs\gputest\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\Anaconda3\envs\gputest\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\User\Anaconda3\envs\gputest\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
train = pd.read_csv('train.csv')
data0 = pd.read_feather('train_data_0.feather')
data1 = pd.read_feather('train_data_1.feather')
data2 = pd.read_feather('train_data_2.feather')
data3 = pd.read_feather('train_data_3.feather')

In [3]:
data_full = pd.concat([data0,data1,data2,data3],ignore_index=True)

In [10]:
class GraphemeDataset(Dataset):
    def __init__(self,df,label,_type='train'):
        self.df = df
        self.label = label
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        label1 = self.label.vowel_diacritic.values[idx]
        label2 = self.label.grapheme_root.values[idx]
        label3 = self.label.consonant_diacritic.values[idx]
        image = self.df.iloc[idx][1:].values.reshape(64,64).astype(np.float)
        return image,label1,label2,label3

In [5]:
class DenseBasicBlock(nn.Module):
    def __init__(self, inplanes, expansion=1, growthRate=32, dropRate=0):
        super(DenseBasicBlock, self).__init__()
        planes = expansion * growthRate
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, growthRate, kernel_size=3,
                               padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        if self.dropRate > 0:
            out = F.dropout(out, p=self.dropRate, training=self.training)

        out = torch.cat((x, out), 1)

        return out


class DenseBottleneck(nn.Module):
    def __init__(self, inplanes, expansion=4, growthRate=32, dropRate=0):
        super(DenseBottleneck, self).__init__()
        planes = expansion * growthRate
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, growthRate, kernel_size=3,
                               padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.dropRate = dropRate

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        if self.dropRate > 0:
            out = F.dropout(out, p=self.dropRate, training=self.training)

        out = torch.cat((x, out), 1)

        return out


class Transition(nn.Module):
    def __init__(self, inplanes, outplanes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(inplanes, outplanes, kernel_size=1, stride=1,
                               bias=False)
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        out = self.bn(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.avgpool(out)
        return out


In [6]:
class DenseNet(nn.Module):
    def __init__(self, bottleneck=True, growthRate=32, head7x7=True, dropRate=0,
                 increasingRate=1, compressionRate=2, layers=(6, 12, 32, 32)):
        """
        For constructuing multiple densenet variants change 'layers' argument to :
        (2, 2, 2, 2) for densenet21 # growthRate=24
        (3, 4, 6, 3) for densenet37 # growthRate=24
        (4, 6, 8, 4) for densenet49 # growthRate=24
        (4, 8, 16, 8) for densenet77
        (6, 12, 24, 16) for densenet121
        (6, 12, 32, 32) for densenet169
        (6, 12, 48, 32) for densenet201
        (6, 12, 64, 48) for densenet264
        (6, 12, 36, 24) for densenet161 # growthRate=48
        (6, 12, 64, 48) for densenet264_g48 # growthRate=48
        note: if you use head7x7=False, the actual depth of densenet will increase by 2 layers.
        """
        
        super(DenseNet, self).__init__()
        if bottleneck:
            block = DenseBottleneck
        else:
            block = DenseBasicBlock

        self.growthRate = growthRate
        self.dropRate = dropRate
        self.increasingRate = increasingRate
        headplanes = growthRate * pow(increasingRate, 2)
        self.inplanes = headplanes * 2  # default 64

        self.head7x7 = head7x7
        if self.head7x7:
            self.conv1 = nn.Conv2d(1, headplanes * 2, 7, 2, 3, bias=False)
            self.bn1 = nn.BatchNorm2d(headplanes * 2)
        else:
            self.conv1 = nn.Conv2d(1, headplanes, 3, 2, 1, bias=False)
            self.bn1 = nn.BatchNorm2d(headplanes)
            self.conv2 = nn.Conv2d(headplanes, headplanes, 3, 1, 1, bias=False)
            self.bn2 = nn.BatchNorm2d(headplanes)
            self.conv3 = nn.Conv2d(headplanes, headplanes * 2, 3, 1, 1, bias=False)
            self.bn3 = nn.BatchNorm2d(headplanes * 2)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Dense-Block 1 and transition (56x56)
        self.dense1 = self._make_layer(block, layers[0])
        self.trans1 = self._make_transition(compressionRate)
        # Dense-Block 2 and transition (28x28)
        self.dense2 = self._make_layer(block, layers[1])
        self.trans2 = self._make_transition(compressionRate)
        # Dense-Block 3 and transition (14x14)
        self.dense3 = self._make_layer(block, layers[2])
        self.trans3 = self._make_transition(compressionRate)
        # Dense-Block 4 (14x14)
        self.dense4 = self._make_layer(block, layers[3])

        self.bn = nn.BatchNorm2d(self.inplanes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        # vowel_diacritic
        self.fc1 = nn.Linear(self.inplanes,11)
        # grapheme_root
        self.fc2 = nn.Linear(self.inplanes,168)
        # consonant_diacritic
        self.fc3 = nn.Linear(self.inplanes,7)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, blocks):
        """ Stack n bottleneck modules where n is inferred from the depth of the network.
        Args:
            block: block type used to construct DenseNet
            blocks: number of blocks to be built
        Returns: a Module consisting of n sequential bottlenecks.
        """
        layers = []
        for i in range(blocks):
            layers.append(block(self.inplanes, growthRate=self.growthRate, dropRate=self.dropRate))
            self.inplanes += self.growthRate

        return nn.Sequential(*layers)

    def _make_transition(self, compressionRate):
        inplanes = self.inplanes
        outplanes = int(math.floor(self.inplanes // compressionRate))
        self.inplanes = outplanes
        self.growthRate *= self.increasingRate
        return Transition(inplanes, outplanes)

    def forward(self, x):
        if self.head7x7:
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
        else:
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.conv2(x)
            x = self.bn2(x)
            x = self.relu(x)
            x = self.conv3(x)
            x = self.bn3(x)
            x = self.relu(x)
        x = self.maxpool(x)
        x = self.trans1(self.dense1(x))
        x = self.trans2(self.dense2(x))
        x = self.trans3(self.dense3(x))
        x = self.dense4(x)
        x = self.bn(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x3 = self.fc3(x)
        return x1,x2,x3

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
#model = ResNet50().to(device)
model=DenseNet().to(device)
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=4e-4)
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=0.05)
criterion = nn.CrossEntropyLoss()
batch_size=32

In [11]:
#training + validation
epochs = 3
model.train()
losses = []
accs = []
val_losses = []
val_accs = []
for epoch in range(epochs):
    train_index =train.groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']).apply(lambda x: x.sample(5)).image_id.values
    reduced_train = train.loc[train.image_id.isin(train_index)]
    train_data = data_full.loc[data_full.image_id.isin(train_index)]
    
    train_image = GraphemeDataset(train_data,reduced_train)
    
    
    ##data for training
    train_loader = torch.utils.data.DataLoader(train_image,batch_size=batch_size,shuffle=True)
    
    test_index =train.groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']).apply(lambda x: x.sample(1)).image_id.values
    reduced_test = train.loc[train.image_id.isin(test_index)]
    test_data = data_full.loc[data_full.image_id.isin(test_index)]
    
    test_image = GraphemeDataset(test_data,reduced_test)
    
    ##data for test
    test_loader = torch.utils.data.DataLoader(test_image,batch_size=batch_size,shuffle=True)
    
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    for idx, (inputs,labels1,labels2,labels3) in tqdm(enumerate(train_loader),total=len(train_loader)):
        inputs = inputs.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        
        optimizer.zero_grad()
        outputs1,outputs2,outputs3 = model(inputs.unsqueeze(1).float())
        loss1 = criterion(outputs1,labels1)
        loss2 = criterion(outputs2,labels2)
        loss3 = criterion(outputs3,labels3)
        running_loss += loss1+loss2+loss3
        running_acc += (outputs1.argmax(1)==labels1).float().mean()
        running_acc += (outputs2.argmax(1)==labels2).float().mean()
        running_acc += (outputs3.argmax(1)==labels3).float().mean()
        (loss1+loss2+loss3).backward()
        optimizer.step()
    #scheduler.step()
    losses.append(running_loss/len(train_loader))
    accs.append(running_acc/(len(train_loader)*3))
    print('acc : {:.4f}%'.format(running_acc*100/(len(train_loader)*3)))
    print('loss : {:.4f}'.format(running_loss/len(train_loader)))
    
    ## data feed for validation
    with torch.no_grad():
        running_loss = 0.0
        running_acc = 0.0
        for idx, (inputs,labels1,labels2,labels3) in tqdm(enumerate(test_loader),total=len(test_loader)):  #here tqdm is used for progressbar and total=len(t..) means prpgress bar highest value is len(t..)
            
            inputs = inputs.to(device)
            labels1 = labels1.to(device)
            labels2 = labels2.to(device)
            labels3 = labels3.to(device)
            
            outputs1,outputs2,outputs3 = model(inputs.unsqueeze(1).float())
            
            loss1 = criterion(outputs1,labels1)
            loss2 = criterion(outputs2,labels2)
            loss3 = criterion(outputs3,labels3)
            running_loss += loss1+loss2+loss3
            running_acc += (outputs1.argmax(1)==labels1).float().mean()
            running_acc += (outputs2.argmax(1)==labels2).float().mean()
            running_acc += (outputs3.argmax(1)==labels3).float().mean()
            
        
        val_losses.append(running_loss/len(test_loader))
        val_accs.append(running_acc/(len(test_loader)*3))
        print('val_acc : {:.4f}%'.format(running_acc*100/(len(test_loader)*3)))
        print('va_loss : {:.4f}'.format(running_loss/len(test_loader)))
            
torch.save(model.state_dict(), 'Densesaved_weightsTrained50.pth')

epochs 1/3 



acc : 38.3509%
loss : 7.4335



val_acc : 49.4072%
va_loss : 6.0458
epochs 2/3 



acc : 53.3740%
loss : 5.4795



val_acc : 56.7412%
va_loss : 4.8611
epochs 3/3 



acc : 58.3223%
loss : 4.5914



val_acc : 62.4238%
va_loss : 4.0513


INFERENCE PART

In [17]:
test = pd.read_csv('test.csv')

In [18]:
class GraphemeDataset(Dataset):
    def __init__(self,df,_type='train'):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        image = self.df.iloc[idx][1:].values.reshape(64,64).astype(float)
        return image

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = DenseNet().to(device)
model.load_state_dict(torch.load('dense_saved_weights.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [20]:
def Resize(df,size=64):
    resized = {} 
    df = df.set_index('image_id')
    for i in tqdm(range(df.shape[0])):
        image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T.reset_index()
    resized.columns = resized.columns.astype(str)
    resized.rename(columns={'index':'image_id'},inplace=True)
    return resized

In [21]:
import cv2
model.eval()
test_data = ['test_image_data_0.parquet','test_image_data_1.parquet','test_image_data_2.parquet','test_image_data_3.parquet']
predictions = []
batch_size=1
for fname in test_data:
    data = pd.read_parquet(f'{fname}')
    data = Resize(data)
    test_image = GraphemeDataset(data)
    test_loader = torch.utils.data.DataLoader(test_image,batch_size=1,shuffle=False)
    with torch.no_grad():
        for idx, (inputs) in tqdm(enumerate(test_loader),total=len(test_loader)):
            inputs.to(device)
            
            outputs1,outputs2,outputs3 = model(inputs.unsqueeze(1).float().cuda())
            predictions.append(outputs3.argmax(1).cpu().detach().numpy())
            predictions.append(outputs2.argmax(1).cpu().detach().numpy())
            predictions.append(outputs1.argmax(1).cpu().detach().numpy())

In [22]:
submission = pd.read_csv('sample_submission.csv')
submission.target = np.hstack(predictions)
submission.head(10)

,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,96
2,Test_0_vowel_diacritic,1
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,141
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0


In [ ]:
submission.to_csv('submissiondense.csv',index=False)